# 2D Magnetic Field Diffusion (Rm << 1)

**Validation case**: 2D magnetic field diffusion in the resistive limit

## Learning Objectives

After completing this notebook, you will understand:

1. The magnetic induction equation and the role of resistivity
2. The magnetic Reynolds number and what Rm << 1 means physically
3. How a 2D Gaussian magnetic field profile spreads due to diffusion
4. The analytic solution for 2D magnetic diffusion
5. Coordinate transformation: cylindrical (r,z) approximating Cartesian (x,z)

---
## 1. Physics Background

### The Magnetic Induction Equation

The evolution of magnetic field in a conducting plasma is governed by the **induction equation**:

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B}) + \eta \nabla^2 \mathbf{B}$$

where:
- $\mathbf{v}$ is the plasma velocity
- $\eta = 1/(\mu_0 \sigma)$ is the magnetic diffusivity (related to resistivity)
- $\sigma$ is the electrical conductivity

The two terms represent competing physics:

| Term | Physics | Effect |
|------|---------|--------|
| $\nabla \times (\mathbf{v} \times \mathbf{B})$ | Advection | Field frozen to plasma, moves with flow |
| $\eta \nabla^2 \mathbf{B}$ | Diffusion | Field slips through plasma, spreads out |

### The Magnetic Reynolds Number

The **magnetic Reynolds number** measures the relative importance of advection vs diffusion:

$$R_m = \frac{v L}{\eta} = \frac{\text{advection rate}}{\text{diffusion rate}}$$

where $v$ is a characteristic velocity and $L$ is a characteristic length scale.

| Regime | Condition | Physics |
|--------|-----------|--------|
| **Ideal MHD** | $R_m \gg 1$ | Advection dominates, field frozen to plasma |
| **Resistive MHD** | $R_m \ll 1$ | Diffusion dominates, field slips through plasma |

In this notebook, we study the **resistive limit** ($R_m \ll 1$) where $\mathbf{v} = 0$ and the induction equation reduces to:

$$\frac{\partial \mathbf{B}}{\partial t} = \eta \nabla^2 \mathbf{B}$$

This is the **heat equation** (or diffusion equation) for the magnetic field!

### Analytic Solution: 2D Spreading Gaussian

For a 2D Gaussian initial condition centered at $(x_0, z_0)$:

$$B_z(x, z, t=0) = B_{\text{peak}} \exp\left(-\frac{(x-x_0)^2 + (z-z_0)^2}{2\sigma_0^2}\right)$$

The exact solution is a **spreading Gaussian** in 2D:

$$B_z(x, z, t) = B_{\text{peak}} \frac{\sigma_0^2}{\sigma_t^2} \exp\left(-\frac{(x-x_0)^2 + (z-z_0)^2}{2\sigma_t^2}\right)$$

where $\sigma_t^2 = \sigma_0^2 + 2Dt$ and $D = \eta/\mu_0$ is the magnetic diffusivity.

Key features:
- **Width grows**: $\sigma(t) = \sqrt{\sigma_0^2 + 2Dt}$ (same in both directions)
- **Peak decreases**: amplitude $\propto \sigma_0^2/\sigma_t^2$ in 2D (not $1/\sigma$ like 1D)
- **Characteristic timescale**: $\tau_{\text{diff}} = \sigma_0^2 / (2D)$

### Coordinate Transformation

The MHD code uses **cylindrical coordinates (r, z)** while the analytic solution is for **Cartesian (x, z)**. 

When $r \gg \sigma$ (far from axis), the cylindrical Laplacian approximates Cartesian:

$$\nabla^2 B_z \approx \frac{\partial^2 B_z}{\partial r^2} + \frac{\partial^2 B_z}{\partial z^2}$$

The $(1/r)\partial B_z/\partial r$ term becomes negligible. We map $x = r - r_{\text{center}}$.

### Physical Intuition

Magnetic diffusion is analogous to heat conduction:

- A localized "hot spot" of magnetic field spreads into surrounding regions
- The field diffuses from high-B to low-B regions
- Higher resistivity ($\eta$) means faster diffusion
- The process is irreversible (entropy increases)

In fusion plasmas, magnetic diffusion is usually slow ($R_m \sim 10^6$ in tokamaks), but it becomes important:
- During magnetic reconnection events
- In resistive instabilities (tearing modes)
- Near plasma boundaries where temperature (and thus conductivity) is lower

---
## 2. Configuration Setup

In [ ]:
# Standard imports
import jax
import jax.numpy as jnp
import jax.lax as lax
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

# jax-frc imports
from jax_frc.configurations import MagneticDiffusionConfiguration
from jax_frc.solvers.explicit import EulerSolver

# Notebook utilities
import sys
sys.path.insert(0, '.')
from _shared import plot_comparison, plot_error, compute_metrics, metrics_summary, animate_overlay

# Plotting style
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

In [ ]:
# === PHYSICS PARAMETERS ===

B_peak = 1.0       # Peak magnetic field [T]
sigma = 0.1        # Initial Gaussian width [m] (should be << r_min)

# Resistivity (not diffusivity!) - diffusivity D = eta/mu_0
eta = 1.26e-10     # Magnetic resistivity [Ω·m]
MU0 = 1.2566e-6    # Permeability of free space
diffusivity = eta / MU0  # ~1e-4 m²/s

print(f"Resistivity η = {eta:.2e} Ω·m")
print(f"Diffusivity D = η/μ₀ = {diffusivity:.2e} m²/s")

# === GRID PARAMETERS (2D slab geometry) ===

nr = 64            # Radial resolution  
nz = 64            # Axial resolution
r_min = 1.0        # Inner radius [m] - far from axis for Cartesian approx
r_max = 2.0        # Outer radius [m]
z_extent = 1.0     # Domain: z ∈ [-z_extent, z_extent]

# Center of Gaussian in slab
r_center = 1.5     # Radial center [m] = (r_min + r_max) / 2
z_center = 0.0     # Axial center [m]

print(f"\nSlab geometry: r ∈ [{r_min}, {r_max}], z ∈ [-{z_extent}, {z_extent}]")
print(f"Gaussian center: (r={r_center}, z={z_center})")
print(f"Coordinate approx. error: σ/r_center = {sigma/r_center:.1%}")

# === TIME PARAMETERS ===

# Diffusion timescale (using diffusivity, not eta)
tau_diff = sigma**2 / (2 * diffusivity)
print(f"\nDiffusion timescale: τ_diff = {tau_diff:.2f} s")

t_end = 0.1 * tau_diff    # Run for 0.1 diffusion times

# CFL constraint for diffusion
dr = (r_max - r_min) / nr
dz = 2 * z_extent / nz
dx_min = min(dr, dz)
dt_max = 0.25 * dx_min**2 / diffusivity
dt = dt_max * 0.5  # Safety factor

print(f"Simulation time: t_end = {t_end:.2f} s")
print(f"Timestep: dt = {dt:.4f} s (CFL: {dt_max:.4f} s)")
print(f"Expected steps: ~{int(t_end/dt)}")

In [ ]:
# Create configuration with 2D slab geometry
config = MagneticDiffusionConfiguration(
    B_peak=B_peak,
    sigma=sigma,
    eta=eta,
    nr=nr,
    nz=nz,
    r_min=r_min,
    r_max=r_max,
    z_extent=z_extent,
    r_center=r_center,
    z_center=z_center,
)

# Build simulation components
geometry = config.build_geometry()
initial_state = config.build_initial_state(geometry)
model = config.build_model()
solver = EulerSolver()

print(f"Grid: {geometry.nr} × {geometry.nz} (r × z)")
print(f"Domain: r ∈ [{geometry.r_min:.1f}, {geometry.r_max:.1f}], z ∈ [{geometry.z_min:.1f}, {geometry.z_max:.1f}]")
print(f"Resolution: Δr = {geometry.dr:.4f}, Δz = {geometry.dz:.4f}")
print(f"\nModel: {type(model).__name__}")
print(f"Magnetic Reynolds number: Rm = {config.magnetic_reynolds_number():.2e} (should be << 1)")

### Visualize 2D Initial Condition

In [ ]:
# Get 2D grid coordinates
r = np.array(geometry.r_grid)
z = np.array(geometry.z_grid)

# Initial B_z profile (2D)
Bz_init = np.array(initial_state.B[:, :, 2])

# Create figure with 2D contour and cross-sections
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 2D contour plot
im = axes[0].contourf(z, r, Bz_init, levels=20, cmap='RdBu_r')
axes[0].set_xlabel('z [m]', fontsize=12)
axes[0].set_ylabel('r [m]', fontsize=12)
axes[0].set_title('Initial $B_z$ (2D)', fontsize=14)
plt.colorbar(im, ax=axes[0], label='$B_z$ [T]')
# Mark center
axes[0].plot(z_center, r_center, 'k+', markersize=15, markeredgewidth=2)

# Radial slice at z=0
z_mid_idx = geometry.nz // 2
r_1d = r[:, z_mid_idx]
Bz_r = Bz_init[:, z_mid_idx]
axes[1].plot(r_1d, Bz_r, 'b-', linewidth=2)
axes[1].axvline(r_center, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('r [m]', fontsize=12)
axes[1].set_ylabel('$B_z$ [T]', fontsize=12)
axes[1].set_title('Radial slice at z=0', fontsize=14)
axes[1].grid(True, alpha=0.3)

# Axial slice at r=r_center
r_mid_idx = geometry.nr // 2
z_1d = z[r_mid_idx, :]
Bz_z = Bz_init[r_mid_idx, :]
axes[2].plot(z_1d, Bz_z, 'b-', linewidth=2)
axes[2].axvline(z_center, color='gray', linestyle='--', alpha=0.5)
axes[2].set_xlabel('z [m]', fontsize=12)
axes[2].set_ylabel('$B_z$ [T]', fontsize=12)
axes[2].set_title('Axial slice at r=r_center', fontsize=14)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Peak B_z = {np.max(Bz_init):.3f} T at (r={r_center}, z={z_center})")
print(f"Initial width σ₀ = {sigma} m")

---
## 3. Run 2D Simulation

We evolve the 2D Gaussian profile forward in time. The magnetic field should spread isotropically according to the analytic solution.

In [ ]:
# Time stepping setup
n_steps = int(t_end / dt)
save_interval = max(1, n_steps // 50)  # ~50 snapshots

print(f"Running {n_steps} steps with dt = {dt:.2e}")
print(f"Saving every {save_interval} steps")

In [ ]:
# Run simulation using lax.scan for efficiency
@jax.jit
def scan_step(state, _):
    new_state = solver.step(state, dt, model, geometry)
    return new_state, new_state.B[:, :, 2]  # Save full 2D B_z

# Run simulation
print("Running 2D simulation...")
final_state, Bz_history = lax.scan(scan_step, initial_state, None, length=n_steps)

# Convert to numpy and subsample for plotting
Bz_history = np.array(Bz_history)[::save_interval]
times = np.arange(0, n_steps, save_interval) * dt

# Add final state if not included
Bz_final = np.array(final_state.B[:, :, 2])
if len(times) == 0 or times[-1] < t_end - dt:
    Bz_history = np.concatenate([Bz_history, Bz_final[np.newaxis, :, :]], axis=0)
    times = np.append(times, final_state.time)

print(f"Simulation complete: t = {final_state.time:.4f} s")
print(f"Saved {len(times)} 2D snapshots")

---
## 4. Compare with 2D Analytic Solution

In [ ]:
# Compute 2D analytic solution at final time
Bz_analytic = np.array(config.analytic_solution(jnp.array(r), jnp.array(z), final_state.time))

# Expected width at final time
sigma_final = np.sqrt(sigma**2 + 2 * diffusivity * final_state.time)
print(f"Initial width: σ₀ = {sigma:.4f} m")
print(f"Final width:   σ(t) = {sigma_final:.4f} m")
print(f"Width increase: {(sigma_final/sigma - 1)*100:.1f}%")
print(f"\nPeak decay: σ₀²/σ_t² = {(sigma/sigma_final)**2:.3f} (2D scaling)")

In [ ]:
# 2D comparison plots
fig, axes = plt.subplots(2, 3, figsize=(15, 9))

# Top row: Initial, Final Numerical, Final Analytic (2D contours)
vmax = B_peak
levels = np.linspace(0, vmax, 21)

im0 = axes[0, 0].contourf(z, r, Bz_init, levels=levels, cmap='RdBu_r')
axes[0, 0].set_title('Initial $B_z$', fontsize=12)
axes[0, 0].set_ylabel('r [m]')
plt.colorbar(im0, ax=axes[0, 0])

im1 = axes[0, 1].contourf(z, r, Bz_final, levels=levels, cmap='RdBu_r')
axes[0, 1].set_title(f'Numerical $B_z$ at t={final_state.time:.2f}s', fontsize=12)
plt.colorbar(im1, ax=axes[0, 1])

im2 = axes[0, 2].contourf(z, r, Bz_analytic, levels=levels, cmap='RdBu_r')
axes[0, 2].set_title(f'Analytic $B_z$ at t={final_state.time:.2f}s', fontsize=12)
plt.colorbar(im2, ax=axes[0, 2])

# Bottom row: Error, Radial slice, Axial slice
error_2d = Bz_final - Bz_analytic
err_max = np.max(np.abs(error_2d))
im3 = axes[1, 0].contourf(z, r, error_2d, levels=20, cmap='coolwarm')
axes[1, 0].set_title('Error (Num - Ana)', fontsize=12)
axes[1, 0].set_xlabel('z [m]')
axes[1, 0].set_ylabel('r [m]')
plt.colorbar(im3, ax=axes[1, 0], label='Error [T]')

# Radial slice comparison
axes[1, 1].plot(r_1d, Bz_init[:, z_mid_idx], 'gray', linestyle=':', label='Initial')
axes[1, 1].plot(r_1d, Bz_final[:, z_mid_idx], 'b-', linewidth=2, label='Numerical')
axes[1, 1].plot(r_1d, Bz_analytic[:, z_mid_idx], 'r--', linewidth=2, label='Analytic')
axes[1, 1].set_xlabel('r [m]')
axes[1, 1].set_ylabel('$B_z$ [T]')
axes[1, 1].set_title('Radial slice at z=0')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Axial slice comparison
axes[1, 2].plot(z_1d, Bz_init[r_mid_idx, :], 'gray', linestyle=':', label='Initial')
axes[1, 2].plot(z_1d, Bz_final[r_mid_idx, :], 'b-', linewidth=2, label='Numerical')
axes[1, 2].plot(z_1d, Bz_analytic[r_mid_idx, :], 'r--', linewidth=2, label='Analytic')
axes[1, 2].set_xlabel('z [m]')
axes[1, 2].set_ylabel('$B_z$ [T]')
axes[1, 2].set_title('Axial slice at r=r_center')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Error distribution (use shared utility on flattened arrays)
fig, axes = plot_error(
    np.arange(Bz_final.size), Bz_final.ravel(), Bz_analytic.ravel(),
    xlabel='Grid point index',
    title='2D Error Analysis (flattened)'
)
plt.show()

In [ ]:
# Compute validation metrics on full 2D field
metrics = compute_metrics(Bz_final.ravel(), Bz_analytic.ravel())

# Display with thresholds (adjusted for coordinate transformation approximation)
thresholds = {
    'l2_error': 0.10,       # 10% relative L2 error (higher due to cylindrical approx)
    'max_rel_error': 0.15,  # 15% max relative error
}

print("2D Validation Metrics:")
print("=" * 50)
for name, value in metrics.items():
    threshold = thresholds.get(name, None)
    if threshold:
        status = '✓ PASS' if value <= threshold else '✗ FAIL'
        print(f"{name:20s}: {value:.4e}  (threshold: {threshold:.2e}) {status}")
    else:
        print(f"{name:20s}: {value:.4e}")

# Overall result
overall_pass = all(metrics[k] <= thresholds[k] for k in thresholds if k in metrics)
print("=" * 50)
print(f"Overall: {'✓ PASS' if overall_pass else '✗ FAIL'}")
print(f"\nNote: ~{sigma/r_center:.0%} error expected from cylindrical→Cartesian approximation")

---
## 5. 2D Time Evolution Animation

In [ ]:
# 2D animation of spreading Gaussian
from matplotlib.animation import FuncAnimation

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Initialize plots
levels = np.linspace(0, B_peak, 21)
im_num = axes[0].contourf(z, r, Bz_history[0], levels=levels, cmap='RdBu_r')
axes[0].set_xlabel('z [m]')
axes[0].set_ylabel('r [m]')
title_num = axes[0].set_title('Numerical')
plt.colorbar(im_num, ax=axes[0], label='$B_z$ [T]')

# Analytic at t=0
Bz_ana_0 = np.array(config.analytic_solution(jnp.array(r), jnp.array(z), 0.0))
im_ana = axes[1].contourf(z, r, Bz_ana_0, levels=levels, cmap='RdBu_r')
axes[1].set_xlabel('z [m]')
axes[1].set_ylabel('r [m]')
title_ana = axes[1].set_title('Analytic')
plt.colorbar(im_ana, ax=axes[1], label='$B_z$ [T]')

def animate(frame):
    t = times[frame]
    
    # Clear and replot (contourf doesn't support set_array)
    axes[0].clear()
    axes[0].contourf(z, r, Bz_history[frame], levels=levels, cmap='RdBu_r')
    axes[0].set_xlabel('z [m]')
    axes[0].set_ylabel('r [m]')
    axes[0].set_title(f'Numerical (t={t:.3f}s)')
    
    axes[1].clear()
    Bz_ana_t = np.array(config.analytic_solution(jnp.array(r), jnp.array(z), t))
    axes[1].contourf(z, r, Bz_ana_t, levels=levels, cmap='RdBu_r')
    axes[1].set_xlabel('z [m]')
    axes[1].set_ylabel('r [m]')
    axes[1].set_title(f'Analytic (t={t:.3f}s)')
    
    return []

anim = FuncAnimation(fig, animate, frames=len(times), interval=200, blit=False)
plt.close()

HTML(anim.to_jshtml())

---
## 6. Interactive 2D Exploration

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def explore_time_2d(time_idx):
    """Show 2D solution at a specific snapshot."""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    t = times[time_idx]
    Bz_num = Bz_history[time_idx]
    Bz_ana = np.array(config.analytic_solution(jnp.array(r), jnp.array(z), t))
    
    # 2D numerical
    levels = np.linspace(0, B_peak, 21)
    im0 = axes[0, 0].contourf(z, r, Bz_num, levels=levels, cmap='RdBu_r')
    axes[0, 0].set_xlabel('z [m]')
    axes[0, 0].set_ylabel('r [m]')
    axes[0, 0].set_title(f'Numerical $B_z$ at t = {t:.4f} s')
    plt.colorbar(im0, ax=axes[0, 0])
    
    # 2D analytic
    im1 = axes[0, 1].contourf(z, r, Bz_ana, levels=levels, cmap='RdBu_r')
    axes[0, 1].set_xlabel('z [m]')
    axes[0, 1].set_ylabel('r [m]')
    axes[0, 1].set_title(f'Analytic $B_z$ at t = {t:.4f} s')
    plt.colorbar(im1, ax=axes[0, 1])
    
    # Radial slice at z=0
    axes[1, 0].plot(r_1d, Bz_init[:, z_mid_idx], 'gray', linestyle=':', alpha=0.7, label='Initial')
    axes[1, 0].plot(r_1d, Bz_num[:, z_mid_idx], 'b-', linewidth=2, label='Numerical')
    axes[1, 0].plot(r_1d, Bz_ana[:, z_mid_idx], 'r--', linewidth=2, label='Analytic')
    axes[1, 0].set_xlabel('r [m]')
    axes[1, 0].set_ylabel('$B_z$ [T]')
    axes[1, 0].set_title('Radial slice at z=0')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    axes[1, 0].set_ylim(0, B_peak * 1.1)
    
    # Width indicator
    sigma_t = np.sqrt(sigma**2 + 2 * diffusivity * t)
    axes[1, 0].axvline(r_center - sigma_t, color='green', linestyle='--', alpha=0.5)
    axes[1, 0].axvline(r_center + sigma_t, color='green', linestyle='--', alpha=0.5, 
                       label=f'σ(t) = {sigma_t:.3f}')
    axes[1, 0].legend()
    
    # Axial slice at r=r_center
    axes[1, 1].plot(z_1d, Bz_init[r_mid_idx, :], 'gray', linestyle=':', alpha=0.7, label='Initial')
    axes[1, 1].plot(z_1d, Bz_num[r_mid_idx, :], 'b-', linewidth=2, label='Numerical')
    axes[1, 1].plot(z_1d, Bz_ana[r_mid_idx, :], 'r--', linewidth=2, label='Analytic')
    axes[1, 1].set_xlabel('z [m]')
    axes[1, 1].set_ylabel('$B_z$ [T]')
    axes[1, 1].set_title('Axial slice at r=r_center')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].set_ylim(0, B_peak * 1.1)
    
    plt.tight_layout()
    plt.show()
    
    # Print metrics
    m = compute_metrics(Bz_num.ravel(), Bz_ana.ravel())
    print(f"L2 error: {m['l2_error']:.4e}  |  Max error: {m['linf_error']:.4e}")
    print(f"Current width σ(t) = {sigma_t:.4f} m")

# Time slider
time_slider = widgets.IntSlider(
    value=0, min=0, max=len(times)-1, step=1,
    description='Snapshot:',
    continuous_update=False
)

ui = widgets.VBox([
    widgets.HTML('<h4>2D Time Evolution Explorer</h4>'),
    widgets.HTML('<p>Drag the slider to see the 2D solution at different times. Watch the Gaussian spread isotropically!</p>'),
    time_slider,
])

interactive_output = widgets.interactive_output(explore_time_2d, {'time_idx': time_slider})

display(ui, interactive_output)

---
## Summary

This notebook demonstrated **2D magnetic field diffusion** in a slab geometry:

1. **Physics**: In the resistive limit ($R_m \ll 1$), the magnetic induction equation reduces to a diffusion equation. Magnetic field spreads through the plasma like heat through a conductor.

2. **2D Analytic solution**: A 2D Gaussian initial condition evolves as:
   - Width: $\sigma(t) = \sqrt{\sigma_0^2 + 2Dt}$ (isotropic spreading)
   - Peak: $B_{\text{peak}}(t) = B_0 \cdot \sigma_0^2/\sigma_t^2$ (2D amplitude decay)

3. **Coordinate transformation**: We use cylindrical coordinates $(r, z)$ to approximate Cartesian $(x, z)$ by placing the domain far from the axis ($r \gg \sigma$). The error scales as $\sigma/r_{\text{center}} \approx 7\%$.

4. **Validation**: The numerical solution matches the analytic solution within the specified error tolerance, confirming correct implementation of resistive diffusion in 2D.

### Physical Significance

Magnetic diffusion is important in:
- **Magnetic reconnection**: Field lines can break and reconnect when diffusion is significant
- **Resistive instabilities**: Tearing modes, resistive wall modes
- **Edge physics**: Lower temperature means higher resistivity and faster diffusion

### Try Next

- Decrease `sigma` relative to `r_center` for better Cartesian approximation
- Increase resolution (`nr`, `nz`) to reduce numerical error
- Compare with the **frozen flux** notebook to see the opposite limit ($R_m \gg 1$)